In [15]:
import geopandas
import pandas as pd
from tqdm import tqdm

In [16]:
Airport = pd.read_csv("Airports.csv")
Runways = pd.read_csv("Runways.csv")
Waypoints = pd.read_csv("Waypoints.csv")
Navaids = pd.read_csv("Navaids.csv")
Airways = pd.read_csv("Airways.csv")

In [17]:
import networkx as nx
G = nx.DiGraph()
airway_df = pd.read_csv("Airways.csv")
for _, row in airway_df.iterrows():
    source_node = (row['Source'], row['Src_Lat'], row['Src_Long'])
    dest_node = (row['Dest'], row['Dest_Lat'], row['Dest_Long'])
    G.add_node(source_node, lat=row['Src_Lat'], lon=row['Src_Long'])
    G.add_node(dest_node, lat=row['Dest_Lat'], lon=row['Dest_Long'])
    G.add_edge(source_node, dest_node, weight=row['Distance'], airway=row['Airway'])

In [18]:
from geopy.distance import geodesic

def find_nearest_navaids(airport_icao, nav_aid_df, threshold_nm=20): 
    try:
        airport_lat = Airport[Airport["ICAO"]==airport_icao.upper()]["Latitude"].values[0]
        airport_lon = Airport[Airport["ICAO"]==airport_icao.upper()]["Longitude"].values[0]
        nearest_navaids = []
        for _, row in nav_aid_df.iterrows():
            navaid_lat = row['Latitude']
            navaid_lon = row['Longitude']
            navaid_id = row['Navaid']

            # Calculate distance in kilometers
            distance_km = geodesic((airport_lat, airport_lon), (navaid_lat, navaid_lon)).kilometers
            # Convert distance to nautical miles
            distance_nm = distance_km / 1.852
            if distance_nm <= threshold_nm:
                if len(navaid_id)==3:
                    nearest_navaids.append((navaid_id, distance_nm,navaid_lat,navaid_lon))
        return nearest_navaids
    except: 
        return "Give right input"


def get_Route(src, dest):
    origin_ICAO = src
    dest_ICAO = dest
    try:
        origin_navaids = find_nearest_navaids(origin_ICAO, Navaids)
        dest_navaids = find_nearest_navaids(dest_ICAO, Navaids)
        distance = float('inf')
        f = 0
        for o_nav in origin_navaids:
            for d_nav in dest_navaids:
                source = (o_nav[0], o_nav[2], o_nav[3])
                dest = (d_nav[0], d_nav[2], d_nav[3])
                try:
                    shortest_path = nx.dijkstra_path(G, source=source, target=dest, weight='weight')
                    shortest_path_length = nx.dijkstra_path_length(G, source=source, target=dest, weight='weight')
                    if shortest_path_length < distance:
                        f = 1
                        airways = []
                        route = []
                        distseg = []
                        distance = shortest_path_length
                        for s in shortest_path:
                            route.append(s[0])
                        for u, v in zip(shortest_path[:-1], shortest_path[1:]):
                            airway_name = G[u][v].get('airway', 'Unknown')
                            airways.append(airway_name)
                            distseg.append(G[u][v].get('weight', 'Unknown'))
                except:
                    pass  # Ignore if a route does not exist
        if f == 0:
            return "Route does not exist in the Nav Data"
        return {"Route": route, "Airway": airways, "Distance": distance, "DistanceSegment": distseg, "Coord":[(G.nodes[s]['lat'], G.nodes[s]['lon']) for s in shortest_path]}
    except:
        return "Give correct ICAO"


In [22]:
def summarize_ATCroute(route):
    wpts = route['Route']
    airways = route['Airway']
    finalRoot = []
    finalRoot.append(wpts[0])
    prev_Away=airways[0]
    for i,a in enumerate(airways):
        if a==prev_Away:
            continue
        else:
            finalRoot.append(airways[i-1])
            finalRoot.append(wpts[i])
            prev_Away = a
    finalRoot.append(airways[-1])
    finalRoot.append(wpts[-1])
    print(" ".join(finalRoot))

In [20]:
import folium
from IPython.display import display

def plot_route(route_data):
    waypoints = route_data["Route"]
    waypoint_coords = route_data["Coord"]
    airways = route_data['Airway']
    
    avg_lat = sum(coord[0] for coord in waypoint_coords) / len(waypoint_coords)
    avg_lon = sum(coord[1] for coord in waypoint_coords) / len(waypoint_coords)
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=5, tiles="cartodb positron")

    
    for i, (name, coord) in enumerate(zip(waypoints, waypoint_coords)):
        folium.Marker(
            location=coord,
            popup=name,
            icon=folium.Icon(icon='star', color='green')
        ).add_to(m)
    
    for i in range(len(waypoint_coords) - 1):
        folium.PolyLine(
            locations=[waypoint_coords[i], waypoint_coords[i + 1]],
            color='blue',
            weight=2,
            opacity=0.7,
            popup=f'Airway: {airways[i]}'
        ).add_to(m)

    display(m)

## Lets Check the route Kolkata (VECC) to Almaty (UAAA)

In [28]:
route = get_Route("VECC","UAAA")
print("Waypoints:", route["Route"])
print("\n")
print(f"Total Distance: {route['Distance']:.2f} NM")
print("\n")
print(f"Airways used: {route['Airway']}")
print("\n")
print("ATC route:")
summarize_ATCroute(route)

Waypoints: ['CEA', 'OPESU', 'TEPAL', 'GGC', 'UXUPO', 'GUGIP', 'ASARI', 'SULOM', 'INDEK', 'PUNOP', 'ISDUR', 'RN', 'KASMA', 'TANED', 'SS', 'ONEVI', 'DURKA', 'ATROL', 'GERRY', 'MOTMO', 'FIRUZ', 'VAJEN', 'POMIR', 'ASMAN', 'INLOS', 'OSH', 'IRISU', 'MALEX', 'MEDAS', 'BALVU', 'SOVOP', 'MAMIR', 'LAKEL', 'IZIMA', 'BEDUR', 'DETAK', 'BAKIS', 'ATA']


Total Distance: 1870.39 NM


Airways used: ['R460', 'R460', 'R460', 'L509', 'L509', 'L509', 'L509', 'L509', 'Z306', 'Z306', 'Z306', 'J149', 'J130', 'J130', 'J130', 'J130', 'J130', 'P500', 'P500', 'P500', 'G500', 'G500', 'B496', 'L142', 'L145', 'L135', 'L135', 'L135', 'L135', 'L135', 'L135', 'P984', 'P984', 'L998', 'L998', 'L998', 'W333']


ATC route:
CEA R460 GGC L509 INDEK Z306 RN J149 KASMA J130 ATROL P500 FIRUZ G500 POMIR B496 ASMAN L142 INLOS L145 OSH L135 MAMIR P984 IZIMA L998 BAKIS W333 ATA


In [8]:
plot_route(route)

## Lets Check the Route Kolkata (VECC) to Mumbai (VABB)

In [23]:
route = get_Route("VECC","VABB")
print("Waypoints:", route["Route"])
print("\n")
print(f"Total Distance: {route['Distance']:.2f} NM")
print("\n")
print(f"Airways used: {route['Airway']}")
print("\n")
print("ATC route:")
summarize_ATCroute(route)

Waypoints: ['CEA', 'NUDLA', 'JRS', 'OGUNA', 'IKINA', 'OSVAS', 'OMLEG', 'TASEX', 'ASIPI', 'LUXIP', 'OPAKA', 'BBB']


Total Distance: 900.62 NM


Airways used: ['J46', 'J46', 'W160', 'Z14', 'Z14', 'Q20', 'Q20', 'Q20', 'Q20', 'Q20', 'W18']


ATC route:
CEA J46 JRS W160 OGUNA Z14 OSVAS Q20 OPAKA W18 BBB


In [24]:
plot_route(route)

## Lets Check the Route Mumbai (VABB) to Bangkok (VTBS)

In [29]:
route = get_Route("VABB","VTBS")
print("Waypoints:", route["Route"])
print("\n")
print(f"Total Distance: {route['Distance']:.2f} NM")
print("\n")
print(f"Airways used: {route['Airway']}")
print("\n")
print("ATC route:")
summarize_ATCroute(route)

Waypoints: ['BBB', 'OPAKA', 'MELAX', 'IBELA', 'BUSBO', 'MEPIP', 'MEPOK', 'VVZ', 'POTAS', 'OLSOR', 'DOGEM', 'URKOK', 'GUSTU', 'MABUR', 'RINDA', 'SADUS', 'DWI', 'TANEK', 'PASTO', 'BKK']


Total Distance: 1638.80 NM


Airways used: ['G450', 'W218', 'L505', 'L505', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301', 'L301']


ATC route:
BBB G450 OPAKA W218 MELAX L505 BUSBO L301 BKK


In [30]:
plot_route(route)

## Lets Check the Route Chennai (VOMM) to Delhi (VIDP)

In [31]:
route = get_Route("VOMM","VIDP")
print("Waypoints:", route["Route"])
print("\n")
print(f"Total Distance: {route['Distance']:.2f} NM")
print("\n")
print(f"Airways used: {route['Airway']}")
print("\n")
print("ATC route:")
summarize_ATCroute(route)

Waypoints: ['MMV', 'KAMGU', 'KIKUR', 'ANDAV', 'ALBED', 'LAROB', 'SUDEL', 'TASEX', 'RENAG', 'BPL', 'PUKES', 'BUKLO', 'BAVOX', 'SURGO', 'SAPLO', 'DPN']


Total Distance: 956.19 NM


Airways used: ['W20', 'Q24', 'Q24', 'Q24', 'Q24', 'Q24', 'Q24', 'Q24', 'Q24', 'W20', 'W20', 'W20', 'W20', 'W20', 'W20']


ATC route:
MMV W20 KAMGU Q24 BPL W20 DPN


In [32]:
plot_route(route)